## **RAG의 핵심, 문서 검색기 Retriever**
### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**
**Chroma DB에 문서 벡터 저장**

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

#헌법 PDF 파일 로드
# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))


**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [8]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 국회의원이 가져야 하는 책임은 어떤 것들이 있습니까?', '2. 국회의원은 어떤 의무를 부담해야 하나요?', '3. 국회의원이 수행해야 하는 역할에는 어떤 것들이 포함되어 있나요?']


4

In [9]:
unique_docs

[Document(metadata={'page': 4, 'page_label': '5', 'creationdate': '2025-05-14T15:38:10+09:00', 'creator': 'PyPDF', 'source': '/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'total_pages': 14, 'moddate': '2025-05-14T15:38:10+09:00'}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.'),
 Document(metadata={'creator': 'PyPDF', 'moddate': '2025-05-14T15:38:10+09:00', 'creationdate': '2025-05-14T15:38:10+09:00', 'producer': 'iTe